# Read Me
このノートブックでは交差検証について学びます。

# 課題J-1: 交差検証
## 交差検証のアルゴリズム

1. データを分割して
2. 「一部のデータ」に分析手法を使って（訓練）
3. 残った部分でその分析手法の「良さ」を評価する（テスト）
4. 「一部のデータ」をいろいろ動かして２と３を繰り返す
5. 複数回行ったテスト結果をもとに、分析手法の「良さ」を評価する

<br>

<例>  

1. データを赤、青、緑の３グループに分割してみる。

![](https://mathwords.net/wp-content/uploads/2017/04/kousakakunin1-300x211.png)

2. 赤、青に特定の分析手法（最小二乗法など）を使って、データを表す直線（回帰式）を引く。

<br>


3. 緑が回帰式でどれくらい説明できているのかを評価する。例えば、緑が直線からどれくらい離れているのかを二乗誤差などで評価する。この誤差をegとする。

<br>

4. 同様に、赤と緑で回帰式を新たに作って、それを青で評価したときの誤差をeb、青と緑で回帰式を新たに作って、それを赤で評価したときの誤差を erとする。

<br>

5. 「全体の誤差」として、例えば、誤差の平均$e = \frac{1}{3}(eg+eb+er)$を計算する。eが小さいほど「回帰式の作り方が良い」（汎化性能が高いモデルを作成する分析手法である）と考えることができる。

## ホールドアウトとの違い

<br>

交差検定とホールドアウト法を比べた場合、交差検定の方がより評価結果の信頼が高くなる。なぜならデータを交差して評価するという事を繰り返し、平均化しているので、たまたまテストデータに特定のデータが偏っているという事がないためである。

## メリット・デメリット

<br>

- メリット: 評価結果の信頼性が高い
- デメリット: 多くのテストを必要とするため、時間がかかる



## 実際のデータに対してのKFoldの実装

データの準備

In [4]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
from sklearn.model_selection import KFold
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

腫瘍データを行列X2, 診断データをベクトルy2, に格納

In [7]:
X = cancer.data
y = cancer.target
y = np.array(y)

トレーニング用とテスト用に分割し、検証

In [29]:
from sklearn.linear_model import LogisticRegression
kf = KFold(n_splits=2,shuffle=True,random_state=0)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    logistic_regression = LogisticRegression(C=1.0)

    logistic_regression.fit(X_train, y_train)
    print(f'Train Acc: {logistic_regression.score(X_train, y_train):.3f}')
    print(f'Test Acc: {logistic_regression.score(X_test, y_test):.3f}')

Train Acc: 0.968
Test Acc: 0.954
Train Acc: 0.975
Test Acc: 0.930


# 課題J-2: 交差検証のバリエーション(StratifiedKFold)
## 交差検証のバリエーションStratifiedKFoldの説明

<br>

例えば、正解、不正解の2クラス分類問題で正解の割合が80%、不正解の割合が20%だった時、KFoldはその割合を無視して訓練、テストに分割するが、StratifiedKFoldはその割合を保ったまま、分割する。

![](https://cdn-ak.f.st-hatena.com/images/fotolife/u/upura/20181204/20181204214946.png)

☝の図ではdassの茶色、黄色、青が均等な割合で分かれるように訓練 ( 青 )、テスト ( 茶 ) でデータセットを分割している。

## 実際のデータに対してのStratifiedKFoldの実装

In [31]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=2,shuffle=True,random_state=0)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    logistic_regression = LogisticRegression(C=1.0)

    logistic_regression.fit(X_train, y_train)
    print(f'Train Acc: {logistic_regression.score(X_train, y_train):.3f}')
    print(f'Test Acc: {logistic_regression.score(X_test, y_test):.3f}')

Train Acc: 0.944
Test Acc: 0.954
Train Acc: 0.958
Test Acc: 0.940


# 課題J-2: 交差検証のバリエーション(GroupKFold)
## 交差検証のバリエーションGroupKFoldの説明

<br>

例えば、正解、不正解の2クラス分類問題で正解の割合が80%、不正解の割合が20%だった時、KFoldはその割合を無視して訓練、テストに分割するが、StratifiedKFoldはその割合を保ったまま、分割する。

![](https://cdn-ak.f.st-hatena.com/images/fotolife/u/upura/20181204/20181204214913.png)

☝の図ではdassの茶色、黄色、青の選び方が重ならないように分割している。

## 実際のデータに対してのGroupKFoldの実装

In [38]:
import random
from sklearn.model_selection import GroupKFold
groups = []
for i in range(len(y)):
    a = random.random()
    groups.append(int(round(a, 0)))

group_kfold = GroupKFold(n_splits=2)
for train_index, test_index in group_kfold.split(X, y, groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    logistic_regression = LogisticRegression(C=1.0)

    logistic_regression.fit(X_train, y_train)
    print(f'Train Acc: {logistic_regression.score(X_train, y_train):.3f}')
    print(f'Test Acc: {logistic_regression.score(X_test, y_test):.3f}')

Train Acc: 0.967
Test Acc: 0.952
Train Acc: 0.949
Test Acc: 0.949


おわり